# Objetivo

Realizar o pré-processamento das bases de treino, validação e teste.

# Pacotes

In [1]:
from deltalake import DeltaTable, write_deltalake
import pandas as pd
import numpy as np

import Funcoes

# Leitura da base de dados

Desconsideraremos algumas variáveis analisadas na exploração inicial: Education_Level_v2, Education_Level, CLIENTNUM.

In [2]:
dados = DeltaTable("../0.Base/tmp/dados_segmentados").to_pandas()
dados.drop(['__index_level_0__', 'CLIENTNUM', 'Education_Level_v2', 'Education_Level'], axis=1, inplace=True)
dados.head()

,Customer_Age,Gender,Dependent_count,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,...,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Education_Level_v1,vfm,pmcc,Attrition_Flag,type
0,40,F,3,Married,1.< 40k,Blue,36,6,1,3,...,0.823,4926,85,0.635,0.256,4.Graduate,57.952941,0.054141,Existing Customer,Treino
1,52,M,3,Married,4. >= 80k & < 120k,Blue,33,3,3,4,...,0.426,1427,25,0.667,0.037,3.College,57.080000,0.004285,Existing Customer,Treino
2,57,M,3,Married,4. >= 80k & < 120k,Blue,50,3,2,3,...,0.957,1806,38,0.727,0.412,6.Doctorate,47.526316,0.033282,Existing Customer,Treino
3,48,F,2,Single,1.< 40k,Blue,35,3,3,1,...,0.842,4777,69,0.917,0.567,3.College,69.231884,0.276774,Existing Customer,Treino
4,26,M,0,Single,1.< 40k,Blue,13,4,4,4,...,0.837,2192,36,0.440,0.202,1.Uneducated,60.888889,0.081657,Existing Customer,Treino


In [3]:
# Mapeia a variável target para categórica numérica

lista_target = {
    'Existing Customer': 0,
    'Attrited Customer': 1
}

dados['Attrition_Flag'] = dados['Attrition_Flag'].map(lista_target)
dados['Attrition_Flag'].value_counts()

Attrition_Flag
0    8500
1    1627
Name: count, dtype: int64

In [4]:
# Verificação de algum dado nulo

dados.isnull().sum()

Customer_Age                0
Gender                      0
Dependent_count             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
Education_Level_v1          0
vfm                         0
pmcc                        0
Attrition_Flag              0
type                        0
dtype: int64

In [5]:
dados.columns

Index(['Customer_Age', 'Gender', 'Dependent_count', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Education_Level_v1', 'vfm', 'pmcc', 'Attrition_Flag', 'type'],
      dtype='object')

In [6]:
dados.dtypes

Customer_Age                  int64
Gender                       object
Dependent_count               int64
Marital_Status               object
Income_Category              object
Card_Category                object
Months_on_book                int64
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Credit_Limit                float64
Total_Revolving_Bal           int64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt               int64
Total_Trans_Ct                int64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
Education_Level_v1           object
vfm                         float64
pmcc                        float64
Attrition_Flag                int64
type                         object
dtype: object

## Filtro das bases

In [7]:
dados_treino = dados[dados.type == 'Treino']
dados_val = dados[dados.type == 'Validacao']
dados_teste = dados[dados.type == 'Teste']

# Base de treino

## Variáveis numéricas correlacionadas

As variáveis categóricas numéricas não foram consideradas nessa avaliação. Abaixo aparecem as variáveis com relação monotônica (spearman) com valores acima do limiar de 0.6. Gráfica e numericamente, foi possível notar as relações na Exploracao_Inicial.

Variáveis a serem consideradas: Customer_Age, pmcc, Total_Revolving_Bal e vfm

In [8]:
vars_numericas = ['Customer_Age', 'Months_on_book', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 
                  'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'vfm', 'pmcc']
Funcoes.Vars_Correl(dados_treino, vars_numericas, limiar=0.6)

,Var1,Var2,Valores
0,Customer_Age,Months_on_book,0.767328
1,Credit_Limit,Avg_Open_To_Buy,0.931240
2,Credit_Limit,pmcc,-0.793641
3,Total_Revolving_Bal,Avg_Utilization_Ratio,0.713146
4,Avg_Open_To_Buy,Avg_Utilization_Ratio,-0.679496
5,Avg_Open_To_Buy,pmcc,-0.747020
6,Total_Trans_Amt,Total_Trans_Ct,0.880932
7,Total_Trans_Amt,vfm,0.773730


In [9]:
vars_numericas = ['Customer_Age', 'Total_Revolving_Bal', 'vfm', 'pmcc']
Funcoes.Vars_Correl(dados_treino, vars_numericas, limiar=0.6)

,Var1,Var2,Valores


## Avaliação do IV

O IV (Information Value) é uma técnica que ajuda a avaliar o poder preditivo (separação de classes binárias) das variáveis explicativas (independentes) em relação à variável resposta (dependente), e permite selecionar as variáveis explicativas mais promissoras. O IV está relacionado com o WOE (Weight of Evidence), que é uma outra técnica para avaliar a relação de variáveis independentes e depentente. O WOE nos recorda da regressão logística, uma vez que é calculado com base no logarítmo da odds ou logarítmo da razão de chances.

Link de referência:
- https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html

### Categorização das variáveis numéricas

In [10]:
cortes_idade = Funcoes.Categorizacao(dados_treino, 5, 'Customer_Age')
cortes_Rev_Bal = Funcoes.Categorizacao(dados_treino, 3, 'Total_Revolving_Bal')
cortes_vfm = Funcoes.Categorizacao(dados_treino, 3, 'vfm')
cortes_pmcc = Funcoes.Categorizacao(dados_treino, 3, 'pmcc')

In [11]:
dados_treino = dados_treino.assign(Customer_Age_Cat = pd.cut(dados_treino['Customer_Age'], bins = cortes_idade[1], include_lowest=True))
dados_treino = dados_treino.assign(Total_Revolving_Bal_Cat = pd.cut(dados_treino['Total_Revolving_Bal'], bins = cortes_Rev_Bal[1], include_lowest=True))
dados_treino = dados_treino.assign(vfm_Cat = pd.cut(dados_treino['vfm'], bins = cortes_vfm[1], include_lowest=True))
dados_treino = dados_treino.assign(pmcc_Cat = pd.cut(dados_treino['pmcc'], bins = cortes_pmcc[1], include_lowest=True))

dados_treino = dados_treino.assign(Months_Inactive_12_mon_Cat = dados_treino['Months_Inactive_12_mon'].astype('category'))
dados_treino = dados_treino.assign(Contacts_Count_12_mon_Cat = dados_treino['Contacts_Count_12_mon'].astype('category'))

In [12]:
# Todas as colunas que são categóricas ou objeto

dados_treino.select_dtypes(exclude=['int64','float64']).columns

Index(['Gender', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Education_Level_v1', 'type', 'Customer_Age_Cat',
       'Total_Revolving_Bal_Cat', 'vfm_Cat', 'pmcc_Cat',
       'Months_Inactive_12_mon_Cat', 'Contacts_Count_12_mon_Cat'],
      dtype='object')

### Lista de IVs

In [13]:
# Type é a marcação de base de terino, validação e teste

Funcoes.IV_lista_variaveis(dados_treino.drop(['type'], axis = 1), 'Attrition_Flag')

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Variaveis,IV
10,Contacts_Count_12_mon_Cat,inf
6,Total_Revolving_Bal_Cat,0.756700
9,Months_Inactive_12_mon_Cat,0.365277
8,pmcc_Cat,0.075971
7,vfm_Cat,0.027293
2,Income_Category,0.015827
0,Gender,0.014801
5,Customer_Age_Cat,0.014406
4,Education_Level_v1,0.010977
3,Card_Category,0.002329


In [14]:
# A variável Contacts_Count_12_mon_Cat possui valor - infinito para a última categoria, porque nela temos somente maus. Dessa forma a odds é zero
# E o logarítmo natural de zero é menos infinito
# O WOE está ordenado de forma decrescente

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'Contacts_Count_12_mon_Cat', 'Attrition_Flag')

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Contacts_Count_12_mon_Cat,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,0,0.003404,0.046254,13.587134,2.609123,0.111800,inf
1,1,0.070638,0.161238,2.282583,0.825308,0.074772,inf
2,2,0.242553,0.334853,1.380536,0.322472,0.029764,inf
3,3,0.426383,0.321498,0.754013,-0.282345,0.029614,inf
4,4,0.187234,0.123616,0.660220,-0.415182,0.026413,inf
5,5,0.034894,0.012541,0.359399,-1.023323,0.022874,inf
6,6,0.034894,0.000000,0.000000,-inf,inf,inf


In [15]:
# É importante lembrar da parte Exploracao_Inicial que o saldo rotativo total está inflado em zero
# Aqui notamos inversão do WOE, na terceira categoria

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'Total_Revolving_Bal_Cat', 'Attrition_Flag')

,Total_Revolving_Bal_Cat,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,"(-0.001, 831.0]",0.675745,0.267915,0.396474,-0.925144,0.377301,0.7567
1,"(831.0, 1601.0]",0.118298,0.375081,3.170652,1.153937,0.296312,0.7567
2,"(1601.0, 2517.0]",0.205957,0.357003,1.733384,0.550075,0.083087,0.7567


In [ ]:
# Aqui notamos inversão do WOE, na terceira categoria
# Abaixo identificamos baixa volumetria de churn para categorias baixas

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'Months_Inactive_12_mon_Cat', 'Attrition_Flag')

,Months_Inactive_12_mon_Cat,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,0,0.008511,0.002117,0.248779,-1.391192,0.008894,0.365277
1,1,0.061277,0.245440,4.005440,1.387653,0.255555,0.365277
2,2,0.298723,0.328176,1.098595,0.094032,0.002769,0.365277
3,3,0.519149,0.357980,0.689553,-0.371712,0.059908,0.365277
4,4,0.081702,0.036156,0.442539,-0.815227,0.037130,0.365277
5,5,0.020426,0.017427,0.853183,-0.158782,0.000476,0.365277
6,6,0.010213,0.012704,1.243893,0.218246,0.000544,0.365277


In [18]:
dados_treino[dados_treino['Attrition_Flag'] == 1]['Months_Inactive_12_mon'].value_counts()

Months_Inactive_12_mon
3    610
2    351
4     96
1     72
5     24
6     12
0     10
Name: count, dtype: int64

In [ ]:
# Na parte Exploracao_Inicial observa-se que temos prevalência de bons para valores acima de 20%
# Várias quebras, acima de duas, foram testadas, mas observou-se que a representatividade de churn na última faixa era muito baixa
# Além disso, observou-se pequena inversão do WOE ao longo das faixas

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'pmcc_Cat', 'Attrition_Flag')

,pmcc_Cat,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,"(0.00067, 0.035]",0.385532,0.323453,0.838978,-0.175571,0.010899,0.075971
1,"(0.035, 0.106]",0.385532,0.323127,0.838133,-0.176578,0.011019,0.075971
2,"(0.106, 0.423]",0.228936,0.353420,1.543750,0.434214,0.054053,0.075971


In [20]:
dados_treino[dados_treino['Attrition_Flag'] == 1]['pmcc_Cat'].value_counts()

pmcc_Cat
(0.00067, 0.035]    453
(0.035, 0.106]      453
(0.106, 0.423]      269
Name: count, dtype: int64

In [ ]:
# Observa-se inversão do WOE na terceira categoria, embora a volumetria de churn seja parecida
Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'vfm_Cat', 'Attrition_Flag')

,vfm_Cat,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,"(21.249, 50.688]",0.377021,0.325081,0.862236,-0.148226,0.007699,0.027293
1,"(50.688, 61.466]",0.270638,0.345114,1.275185,0.243092,0.018104,0.027293
2,"(61.466, 190.193]",0.352340,0.329805,0.936040,-0.066098,0.001490,0.027293


In [23]:
dados_treino[dados_treino['Attrition_Flag'] == 1]['vfm_Cat'].value_counts()

vfm_Cat
(21.249, 50.688]     443
(61.466, 190.193]    414
(50.688, 61.466]     318
Name: count, dtype: int64

In [135]:
#lista_vfm_Cat = {
#    '(21.249, 43.333]': '0.(21.249, 50.688]',
#    '(43.333, 50.688]': '0.(21.249, 50.688]',
#    '(50.688, 55.779]': '1.(50.688, 55.779]',
#    '(55.779, 61.466]': '2.(55.779, 61.466]',
#    '(61.466, 75.263]': '3.(61.466, 190.193]',
#    '(75.263, 190.193]': '4.(61.466, 190.193]'
#}
#dados_treino['vfm_Cat_new'] = dados_treino['vfm_Cat'].map(lista_vfm_Cat)

#Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'vfm_Cat_new', 'Attrition_Flag')

#### Ajuste de categorias

In [24]:
lista_contacts = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '>=5',
    6: '>=5'
}
dados_treino['Contacts_Count_12_mon_Cat_new'] = dados_treino['Contacts_Count_12_mon_Cat'].map(lista_contacts)

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'Contacts_Count_12_mon_Cat_new', 'Attrition_Flag')

,Contacts_Count_12_mon_Cat_new,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,0,0.003404,0.046254,13.587134,2.609123,0.111800,0.370626
1,1,0.070638,0.161238,2.282583,0.825308,0.074772,0.370626
2,2,0.242553,0.334853,1.380536,0.322472,0.029764,0.370626
3,3,0.426383,0.321498,0.754013,-0.282345,0.029614,0.370626
4,4,0.187234,0.123616,0.660220,-0.415182,0.026413,0.370626
5,>=5,0.069787,0.012541,0.179699,-1.716470,0.098262,0.370626


In [25]:
cortes_Rev_Bal = Funcoes.Categorizacao(dados_treino, 2, 'Total_Revolving_Bal')

dados_treino = dados_treino.assign(Total_Revolving_Bal_Cat = pd.cut(dados_treino['Total_Revolving_Bal'], bins = cortes_Rev_Bal[1], include_lowest=True))

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'Total_Revolving_Bal_Cat', 'Attrition_Flag')

,Total_Revolving_Bal_Cat,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,"(-0.001, 1262.0]",0.748085,0.453094,0.605672,-0.501416,0.147913,0.376585
1,"(1262.0, 2517.0]",0.251915,0.546906,2.170993,0.775185,0.228672,0.376585


In [26]:
lista_inat = {
    0: '0.<=2',
    1: '0.<=2',
    2: '0.<=2',
    3: '1.3',
    4: '2.>=4',
    5: '2.>=4',
    6: '2.>=4'
}
dados_treino['Months_Inactive_12_mon_Cat_new'] = dados_treino['Months_Inactive_12_mon_Cat'].map(lista_inat)

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'Months_Inactive_12_mon_Cat_new', 'Attrition_Flag')

,Months_Inactive_12_mon_Cat_new,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,0.<=2,0.368511,0.575733,1.562324,0.446174,0.092457,0.176661
1,1.3,0.519149,0.357980,0.689553,-0.371712,0.059908,0.176661
2,2.>=4,0.112340,0.066287,0.590052,-0.527545,0.024295,0.176661


In [27]:
cortes_pmcc = Funcoes.Categorizacao(dados_treino, 2, 'pmcc')

dados_treino = dados_treino.assign(pmcc_Cat = pd.cut(dados_treino['pmcc'], bins = cortes_pmcc[1], include_lowest=True))

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'pmcc_Cat', 'Attrition_Flag')

,pmcc_Cat,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,"(0.00067, 0.0606]",0.55234,0.490065,0.887252,-0.119626,0.007450,0.015561
1,"(0.0606, 0.423]",0.44766,0.509935,1.139113,0.130250,0.008111,0.015561


In [28]:
cortes_vfm = Funcoes.Categorizacao(dados_treino, 2, 'vfm')

dados_treino = dados_treino.assign(vfm_Cat = pd.cut(dados_treino['vfm'], bins = cortes_vfm[1], include_lowest=True))

Funcoes.IV(dados_treino.drop(['type'], axis = 1), 'vfm_Cat', 'Attrition_Flag')

,vfm_Cat,Perc_bads,Perc_bons,Odds,Woe,IV_parcial,IV
0,"(21.249, 55.779]",0.520851,0.496091,0.952463,-0.048704,0.001206,0.002453
1,"(55.779, 190.193]",0.479149,0.503909,1.051675,0.050384,0.001247,0.002453


In [29]:
#Funcoes.IV_lista_variaveis(dados_treino.drop(['type', 'Contacts_Count_12_mon_Cat'], axis = 1), 'Attrition_Flag')